In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

In [4]:
val pensionsDataset = Dataset.fromCache("../cache/pensionsCube.cache")
pensionsDataset.size

pensionsDataset: Dataset = com.github.propi.rdfrules.data.Dataset@4d8cc1cf
res3_1: Int = 2696400

In [5]:
val salariesDataset = Dataset.fromCache("../cache/salariesCube.cache")
salariesDataset.size

salariesDataset: Dataset = com.github.propi.rdfrules.data.Dataset@f300e34
res4_1: Int = 2898

In [6]:
val refPeriodLinking = Dataset("../data/linking/refPeriodLinking.ttl")
refPeriodLinking.size

refPeriodLinking: Dataset = com.github.propi.rdfrules.data.Dataset@41fa2eac
res5_1: Int = 84

In [7]:
val refAreaLinking = Dataset("../data/linking/cssaCZSOLinking.ttl")
refAreaLinking.size

refAreaLinking: Dataset = com.github.propi.rdfrules.data.Dataset@75b0b690
res6_1: Int = 91

In [8]:
val sexLinking = Dataset("../data/linking/sexLinking.ttl")
sexLinking.size

sexLinking: Dataset = com.github.propi.rdfrules.data.Dataset@54379550
res7_1: Int = 5

In [ ]:
val dataset = pensionsDataset + salariesDataset + refPeriodLinking + refAreaLinking + sexLinking
dataset.size

In [ ]:
val index = dataset.index()

In [9]:
val uri = (value: String) => TripleItem.Uri(value)

// salariesMeasures
val averageSalary = TripleItem.Uri("http://data.czso.cz/ontology/prumernaMzda")
val medianSalary = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy")
val oneOfSalariesMeasures = OneOf(averageSalary,medianSalary)

// pensionsMeasures
val averageAge = TripleItem.Uri("https://data.cssz.cz/ontology/measure/prumerny-vek")
val pensionersCount = TripleItem.Uri("https://data.cssz.cz/ontology/measure/pocet-duchodcu")
val averagePensionAmount = TripleItem.Uri("https://data.cssz.cz/ontology/measure/prumerna-vyse-duchodu-v-kc")
val oneOfPensionsMeasures = OneOf(averageAge,pensionersCount,averagePensionAmount)

// cssa dimensions
val cssaDimension = "https://data.cssz.cz/ontology/dimension/"
val cssaRefPeriod = uri(cssaDimension+"refPeriod")
val cssaRefArea = uri(cssaDimension+"refArea")
val cssaSex = uri(cssaDimension+"pohlavi")
val cssaPensionKind = uri(cssaDimension+"druh-duchodu")

// czso dimensions
val czsoOntology = "http://data.czso.cz/ontology/"
val czsoRefPeriod = uri(czsoOntology+"refPeriod")
val czsoRefArea = uri(czsoOntology+"refArea")
val czsoSex = uri(czsoOntology+"sex")

val qbDataSet = uri("http://purl.org/linked-data/cube#dataSet")

val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

uri: String => TripleItem.Uri = ammonite.$sess.cmd8$Helper$$Lambda$2921/0x0000000840d9c040@39090666
averageSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/prumernaMzda"
)
medianSalary: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/medianMzdy"
)
oneOfSalariesMeasures: AtomPattern.AtomItemPattern.OneOf = OneOf(
  ArrayBuffer(
    Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
    Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
  )
)
averageAge: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/measure/prumerny-vek"
)
pensionersCount: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/measure/pocet-duchodcu"
)
averagePensionAmount: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/measure/prumerna-vyse-duchodu-v-kc"
)
oneOfPensionsMeasures: AtomPattern.AtomItemPattern.OneOf = OneOf(
  ArrayBuffer(
    Constant(LongUri("https://data.cssz.cz/ontology/measure/prumerny-vek")),
    Constant(LongUri("https://d

In [ ]:
// takoveto obecné patterny moc nedavaji smysl

val closedDimensions: RulePattern = (
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaSex, `object` = AnyConstant) &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = AnyConstant) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = oneOfPensionsMeasures)
)

In [ ]:
val closedDimensionsTask = Amie()
    .addThreshold(Threshold.MinSupport(5))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(closedDimensions)

In [ ]:
//val closedDimensionsTaskRuleset = index.mine(closedDimensionsTask)
//println(closedDimensionsTaskRuleset.size)

In [ ]:
// export
//closedDimensionsTaskRuleset.export("../rulesets/closedDimensionsTaskRuleset.txt")

In [ ]:
//closedDimensionsTaskRuleset
//.filter(rule => rule.measures(Measure.Support).value > 38)
//.computePcaConfidence(0.5)

//.makeClusters(DbScan(minNeighbours = 3, minSimilarity = 0.99))
//.size
//.foreach(println)



In [ ]:
//val confidenceResultset = closedDimensionsTaskRuleset
//.filter(rule => rule.measures(Measure.Support).value > 38)
//.computePcaConfidence(0.5)

In [ ]:
//confidenceResultset.foreach(println)

In [10]:
val oldAgeTotal = uri("https://data.cssz.cz/resource/pension-kind/PK_old_age_total_S_SI_SRN_ST_SD_2010")
val regionTotalOldAgeTotal = uri("pensions-by-region-total-PK_old_age_total_S_SI_SRN_ST_SD")
val salariesTotal = uri("salaries-total")

oldAgeTotal: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/resource/pension-kind/PK_old_age_total_S_SI_SRN_ST_SD_2010"
)
regionTotalOldAgeTotal: TripleItem.Uri = LongUri(
  "pensions-by-region-total-PK_old_age_total_S_SI_SRN_ST_SD"
)
salariesTotal: TripleItem.Uri = LongUri("salaries-total")

In [ ]:
val oldAgeTotalPattern: RulePattern = (
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = salariesTotal) &:
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = oldAgeTotal) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = regionTotalOldAgeTotal)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount)
)

In [ ]:
val oldAgeTotalPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(9))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(oldAgeTotalPattern)

In [ ]:
val oldAgeTotalPatternTaskRuleset = index.mine(oldAgeTotalPatternTask)
println(oldAgeTotalPatternTaskRuleset.size)

In [ ]:
oldAgeTotalPatternTaskRuleset.export("../rulesets/oldAgeTotalPatternTaskRuleset.txt")

In [ ]:
oldAgeTotalPatternTaskRuleset
//.sortBy(Measure.HeadCoverage, Measure.Lift)
//sort by rule length and then by selected measures
//compute additional measures of significance
//.computeConfidence(0.5)
.computePcaConfidence(0.5)
//.computeLift()
.computeLift()
//.filter(rule => rule.measures(Measure.Lift).value > 1)
.size

In [11]:
val salariesBySex = uri("salaries-by-sex")
val regionBySexOldAgeTotal = uri("pensions-by-region-by-sex-PK_old_age_total_S_SI_SRN_ST_SD")

salariesBySex: TripleItem.Uri = LongUri("salaries-by-sex")
regionBySexOldAgeTotal: TripleItem.Uri = LongUri(
  "pensions-by-region-by-sex-PK_old_age_total_S_SI_SRN_ST_SD"
)

In [ ]:
val oldAgeBySexPattern: RulePattern = (
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'a', predicate = cssaSex, `object` = 'e') &:
    AtomPattern(subject = 'c', predicate = czsoSex, `object` = 'e') &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'd') &:  
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = salariesBySex) &:
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = oldAgeTotal) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = regionBySexOldAgeTotal)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount)
)

In [ ]:
val oldAgeBySexPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(11))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(oldAgeBySexPattern)

In [ ]:
val oldAgeBySexPatternTaskRuleset = index.mine(oldAgeBySexPatternTask)
println(oldAgeBySexPatternTaskRuleset.size)

In [ ]:
oldAgeBySexPatternTaskRuleset.export("../rulesets/oldAgeBySexPatternTaskRuleset.txt")

In [ ]:
oldAgeBySexPatternTaskRuleset
//.filter(rule => rule.measures(Measure.Support).value > 15)
.computeLift(0.0)
//.computeLift()
//.filter(rule => rule.measures(Measure.Lift).value > 1)
//.size
//.makeClusters(DbScan(minNeighbours = 3, minSimilarity = 0.99))
//.size
.foreach(println)

In [ ]:
// wikidata + pensions + salaries

In [ ]:
val wikidataCZSORefAreaLinking = Dataset("../data/linking/czsoWikidataLinking.ttl")
wikidataCZSORefAreaLinking.size

In [ ]:
val wikidataCSSARefAreaLinking = Dataset("../data/linking/cssaWikidataLinking.ttl")
wikidataCSSARefAreaLinking.size

In [ ]:
val wikidata = Dataset.fromCache("../cache/wikidata.cache")

In [ ]:
val wikidataAddedDataset = pensionsDataset + salariesDataset + refPeriodLinking + refAreaLinking + sexLinking + wikidata + wikidataCZSORefAreaLinking + wikidataCSSARefAreaLinking
wikidataAddedDataset.size

In [ ]:
val wikidataAddedIndex = wikidataAddedDataset.index()

In [12]:
val appliesTo = uri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
val wdProperty = (value: Int) => uri("http://www.wikidata.org/prop/P" + value)
val czURI = uri("http://www.wikidata.org/entity/Q213")
val sameAs = uri("http://www.w3.org/2002/07/owl#sameAs")

appliesTo: TripleItem.Uri = LongUri(
  "http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod"
)
wdProperty: Int => TripleItem.Uri = ammonite.$sess.cmd11$Helper$$Lambda$3024/0x0000000840dd9840@5b985637
czURI: TripleItem.Uri = LongUri("http://www.wikidata.org/entity/Q213")
sameAs: TripleItem.Uri = LongUri("http://www.w3.org/2002/07/owl#sameAs")

In [ ]:
val wikiDataOldAgeTotalPattern: RulePattern = (
    
    AtomPattern(subject = 'h', `object` = AnyVariable) &:
    AtomPattern(subject = 'h', predicate = wdProperty(6), `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(102), `object` = 'e') &:  
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = AnyConstant) &:
    AtomPattern(subject = 'e', predicate = appliesTo, `object` = 'd') &:
    
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = salariesTotal) &:
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = oldAgeTotal) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = regionTotalOldAgeTotal)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount)
)

In [ ]:
val wikiDataOldAgeTotalPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(2))
    .addThreshold(Threshold.MaxRuleLength(15))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    //.addConstraint(constantsOnlyAtObject)
    .addPattern(wikiDataOldAgeTotalPattern)

In [ ]:
wikidataAddedIndex.cache("../cache/wikidataAddedIndex.cache")

In [ ]:
val wikiDataOldAgeTotalPatternTaskRuleset = wikidataAddedIndex.mine(wikiDataOldAgeTotalPatternTask)
println(wikiDataOldAgeTotalPatternTaskRuleset.size)

In [ ]:
wikiDataOldAgeTotalPatternTaskRuleset.slice(100,200)foreach(println)

In [ ]:
// yago

In [13]:
val yago = Dataset.fromCache("../cache/yago.cache")
yago.size

yago: Dataset = com.github.propi.rdfrules.data.Dataset@1556bec5
res12_1: Int = 2223455

In [14]:
val yagoCZSOLinking = Dataset("../data/linking/yagoCZSOLinking.ttl")
yagoCZSOLinking.size

yagoCZSOLinking: Dataset = com.github.propi.rdfrules.data.Dataset@185c9ca7
res13_1: Int = 90

In [15]:
val yagoAddedDataset = pensionsDataset + salariesDataset + refPeriodLinking + refAreaLinking + sexLinking + yagoCZSOLinking
yagoAddedDataset.size

yagoAddedDataset: Dataset = com.github.propi.rdfrules.data.Dataset@1d59d662
res14_1: Int = 2699568

In [16]:
val yagoAddedIndex = yagoAddedDataset.index()
yagoAddedIndex.cache("../cache/yagoAddedIndex.cache")

2021-04-19 11:54:33:714 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Predicates trimming.
2021-04-19 11:54:39:757 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects indexing.
2021-04-19 11:54:52:828 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects trimming.
2021-04-19 11:54:53:788 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects indexing.
2021-04-19 11:54:55:927 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects trimming.


yagoAddedIndex: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@255088a9
res15_1: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@255088a9

In [38]:
val yagoOldAgeTotalPattern: RulePattern = (
    AtomPattern(subject = 'b') &:
    AtomPattern(subject = 'c', predicate = oneOfSalariesMeasures, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = czsoRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = salariesTotal) &:
    AtomPattern(subject = 'c', predicate = czsoRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = oldAgeTotal) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = regionTotalOldAgeTotal)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount)
)

yagoOldAgeTotalPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(Variable(Variable(1)), Any, Any, Any),
    AtomPattern(
      Variable(Variable(2)),
      OneOf(
        ArrayBuffer(
          Constant(LongUri("http://data.czso.cz/ontology/prumernaMzda")),
          Constant(LongUri("http://data.czso.cz/ontology/medianMzdy"))
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://data.czso.cz/ontology/refPeriod")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("https://data.cssz.cz/ontology/dimension/refPeriod")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://purl.org/linked-data/cube#dataSet")),
      Constant(LongUri("salaries-total")),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://data.czso.cz/ontology/r

In [39]:
val yagoOldAgeTotalPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(2))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(1))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(yagoOldAgeTotalPattern)

yagoOldAgeTotalPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@1952a770

In [40]:
val yagoOldAgeTotalPatternTaskRuleset = yagoAddedIndex.mine(yagoOldAgeTotalPatternTask)
println(yagoOldAgeTotalPatternTaskRuleset.size)

2021-04-19 12:07:51:841 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=2,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=60000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?b),Any,Any,Any), Mapped(Variable(?c),OneOf(ArrayBuffer(Constant(Constant(1433642368)), Constant(Constant(1396113393)))),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(-901057765)),Variable(?d),Any), Mapped(Variable(?a),Constant(Constant(1169043154)),Variable(?d),Any), Mapped(Variable(?c),Constant(Constant(624690160)),Constant(Constant(79965660)),Any), Mapped(Variable(?c),Constant(Constant(1481837794)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(528884079)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-2096697191)),Constant(Constant(-354818346)),Any), Mapped(Variable(?a),Constant(Constant(624690160)),Constan

0


yagoOldAgeTotalPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@720e5bc4

In [34]:
yagoOldAgeTotalPatternTaskRuleset.export("../rulesets/yagoOldAgeTotalPatternTaskRuleset.txt")